Creación de un proyecto básico en ambiente local
===

* Ultima modificación: Mayo 14, 2022

Diretorio para almacenar el proyecto
---

In [1]:
#
# Crea una carpeta para el proyecto. El proyecto también puede estar alojado en
# un repositorio de GitHub.
#
!rm -rf mlruns
!rm -rf /tmp/wine_prj
!mkdir /tmp/wine_prj

Código en Python
---

In [2]:
%%writefile /tmp/wine_prj/train_elasticnet.py

#
# Puede ejecutarse en la línea de comandos como:
# $ python3 train_elasticnet.py {alpha} {l1_ratio} {verbose}
#

def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y


def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=123456,
    )
    return x_train, x_test, y_train, y_test


def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2


def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")


def run():
    #
    # Entrena un modelo sklearn ElasticNet
    #

    import sys

    from sklearn.linear_model import ElasticNet
    
    import mlflow    

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    alpha = float(sys.argv[1])
    l1_ratio = float(sys.argv[2])
    verbose = int(sys.argv[3])
    
    print('Tracking directory:', mlflow.get_tracking_uri())

    with mlflow.start_run():
    
        estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
        estimator.fit(x_train, y_train)
        mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
        if verbose > 0:
            report(estimator, mse, mae, r2)    
            
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        mlflow.sklearn.log_model(estimator, "model")            


if __name__ == "__main__":
    run()

Writing /tmp/wine_prj/train_elasticnet.py


MLproject
---

In [3]:
%%writefile /tmp/wine_prj/MLproject
name: Proyecto de demostracion

entry_points:
    main:
        parameters:
            alpha: {type: float, default: 0.1}
            l1_ratio: {type: float, default: 0.1}
            verbose: {type: integer, default: 1}
        command: 'python3 train_elasticnet.py {alpha} {l1_ratio} {verbose}'

Writing /tmp/wine_prj/MLproject


Ejecución en el ambiente local con parámetros por defecto
---

In [4]:
#
# Ejecución con parámetros por defecto
#
!mlflow run --env-manager=local /tmp/wine_prj

2022/06/03 22:36:04 INFO mlflow.projects.utils: === Created directory /var/folders/34/8tnnc98d5bv6wy7xzfb0qwhh0000gn/T/tmphbvhg4ba for downloading remote URIs passed to arguments of type 'path' ===
2022/06/03 22:36:04 INFO mlflow.projects.backend.local: === Running command 'python3 train_elasticnet.py 0.1 0.1 1' in run with ID 'b8921438f9d147778129d8939cb36155' === 
Tracking directory: file:///Volumes/GitHub/courses-source/notebooks/mlflow/mlruns
ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=12345):
  MSE: 0.489021012335199
  MAE: 0.551252749110561
  R2: 0.29836649473051535
/Volumes/GitHub/courses-source/notebooks/mlflow/.venv/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
2022/06/03 22:36:08 INFO mlflow.projects: === Run (ID 'b8921438f9d147778129d

Ejecución en el ambiente local con parámetros suministrados por el usuario
---

In [5]:
!mlflow run --env-manager=local /tmp/wine_prj -P alpha=0.2 -P l1_ratio=0.2 -P verbose=1

2022/06/03 22:36:09 INFO mlflow.projects.utils: === Created directory /var/folders/34/8tnnc98d5bv6wy7xzfb0qwhh0000gn/T/tmpkhefbxuo for downloading remote URIs passed to arguments of type 'path' ===
2022/06/03 22:36:09 INFO mlflow.projects.backend.local: === Running command 'python3 train_elasticnet.py 0.2 0.2 1' in run with ID '178b1fea9e3c496988634f61f929a839' === 
Tracking directory: file:///Volumes/GitHub/courses-source/notebooks/mlflow/mlruns
ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.5170837474931838
  MAE: 0.5701436798648394
  R2: 0.2581028767270219
/Volumes/GitHub/courses-source/notebooks/mlflow/.venv/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
2022/06/03 22:36:13 INFO mlflow.projects: === Run (ID '178b1fea9e3c49698863

In [6]:
!mlflow run --env-manager=local /tmp/wine_prj -P alpha=0.1 -P l1_ratio=0.1 -P verbose=1

2022/06/03 22:36:14 INFO mlflow.projects.utils: === Created directory /var/folders/34/8tnnc98d5bv6wy7xzfb0qwhh0000gn/T/tmp9y1qpa3f for downloading remote URIs passed to arguments of type 'path' ===
2022/06/03 22:36:14 INFO mlflow.projects.backend.local: === Running command 'python3 train_elasticnet.py 0.1 0.1 1' in run with ID '57b374a3c7304328867d8a2ce8dc6a6f' === 
Tracking directory: file:///Volumes/GitHub/courses-source/notebooks/mlflow/mlruns
ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=12345):
  MSE: 0.489021012335199
  MAE: 0.551252749110561
  R2: 0.29836649473051535
/Volumes/GitHub/courses-source/notebooks/mlflow/.venv/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
2022/06/03 22:36:18 INFO mlflow.projects: === Run (ID '57b374a3c7304328867d8

In [7]:
!mlflow run --env-manager=local /tmp/wine_prj -P alpha=0.5 -P l1_ratio=0.5 -P verbose=1

2022/06/03 22:36:19 INFO mlflow.projects.utils: === Created directory /var/folders/34/8tnnc98d5bv6wy7xzfb0qwhh0000gn/T/tmprblrr40u for downloading remote URIs passed to arguments of type 'path' ===
2022/06/03 22:36:19 INFO mlflow.projects.backend.local: === Running command 'python3 train_elasticnet.py 0.5 0.5 1' in run with ID '9aae0e2983ca41d28af3f8f9ea7dcecc' === 
Tracking directory: file:///Volumes/GitHub/courses-source/notebooks/mlflow/mlruns
ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.6349429447805036
  MAE: 0.6453803508338732
  R2: 0.0890018368226928
/Volumes/GitHub/courses-source/notebooks/mlflow/.venv/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
2022/06/03 22:36:23 INFO mlflow.projects: === Run (ID '9aae0e2983ca41d28af3f8f9ea7dcecc')

MLflow ui
---

Para visualizar la interfase use:

```bash
mlflow ui
```

**Nota:** En docker usar:

```bash
mlflow ui --host 0.0.0.0 
``` 

con: 

http://127.0.0.1:5001


![assets/mlflow-project-1-local-part-0](assets/mlflow-project-1-local-part-0.png)

**Detalles de la corrida**

![assets/mlflow-project-1-local-part-1](assets/mlflow-project-1-local-part-1.png)
![assets/mlflow-project-1-local-part-2](assets/mlflow-project-1-local-part-2.png)
![assets/mlflow-project-1-local-part-3](assets/mlflow-project-1-local-part-3.png)